In [ ]:
!pip install transformers

In [2]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
from torch.optim import Adam

In [ ]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

In [ ]:
device

'cpu'

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/IR_Project4/chitchat_csv.csv')
df.head()

,Unnamed: 0,query,prompt,reply,class
0,0,Hello How are you doing today?,What's one thing your best friend doesn't know...,whats up MD im doing good how are you doing?,0
1,1,"Im alright, I just took a nap. But it was one ...",What's one thing your best friend doesn't know...,oh wow haha so you still feel tired huh?,0
2,2,Yeah,What's one thing your best friend doesn't know...,did you go to bed late?,0
3,3,I have more of a head ache than anything else,What's one thing your best friend doesn't know...,dude thats terrible,0
4,4,"No, Its just adjusting to a time change But en...",What's one thing your best friend doesn't know...,im refining my skills on Sketch App have you h...,0


In [ ]:
indexok = df[ (df['reply'] == 'Ok Bye!')].index
df.drop(indexok , inplace=True)
#74021

In [ ]:
df

,Unnamed: 0,query,prompt,reply,class
0,0,Hello How are you doing today?,What's one thing your best friend doesn't know...,whats up MD im doing good how are you doing?,0
1,1,"Im alright, I just took a nap. But it was one ...",What's one thing your best friend doesn't know...,oh wow haha so you still feel tired huh?,0
2,2,Yeah,What's one thing your best friend doesn't know...,did you go to bed late?,0
3,3,I have more of a head ache than anything else,What's one thing your best friend doesn't know...,dude thats terrible,0
4,4,"No, Its just adjusting to a time change But en...",What's one thing your best friend doesn't know...,im refining my skills on Sketch App have you h...,0
...,...,...,...,...,...
74013,74013,What distance of hike would be your pick? 10 m...,"If money were no object, how would you spend y...",Ha my ideal hike is like 2 miles but 5 mile hi...,0
74014,74014,Mount Everest is totally a 2 mile hike! You go...,"If money were no object, how would you spend y...",Yes! Wait actually I think it's 4 miles but th...,0
74015,74015,Very nice! You are going to have to do it with...,"If money were no object, how would you spend y...","Oh man that's too bad, I'll take lots of pictu...",0
74016,74016,"Hey as long as you have water and time, the hi...","If money were no object, how would you spend y...","That is so true, I hear you can pay like three...",0


In [3]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_tokens(["<reply>:"])
tokenizer.add_tokens(["<sentiment>:"])
tokenizer.add_tokens(["<ner>:"])
tokenizer.add_special_tokens({"pad_token": "<pad>", 
                                "bos_token": "<startofstring>",
                                "eos_token": "<endofstring>"})


3

In [4]:
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

Embedding(50263, 768)

In [ ]:
model = model.to(device)

In [ ]:
prompt = tokenizer("how are you",return_tensors="pt")
encoded = model.generate(**prompt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
tokenizer.decode(encoded[0])

'how are you <sentiment>: <sentiment>: <sentiment>: <sentiment>: <sentiment>: <sentiment>: <sentiment>: <sentiment>: <sentiment>: <sentiment>: <sentiment>: <sentiment>: <sentiment>: <sentiment>: <sentiment>: <sentiment>: <sentiment>:'

In [ ]:
!pip install flair

In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer as sia

In [ ]:

from flair.models import SequenceTagger
model_ner = SequenceTagger.load('ner-ontonotes-fast') #.load('ner')
from flair.data import Sentence
# load tagger
tagger = SequenceTagger.load("flair/ner-english")

# make example sentence
sentence = Sentence("Do you work in mumbai google John and then went to Washington")

# predict NER tags
tagger.predict(sentence)

# print sentence
print(sentence)

# print predicted NER spans
print('The following NER tags are found:')
# iterate over entities and print
for entity in sentence.get_spans('ner'):
    print(entity)


2023-04-02 05:53:57,632 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY
2023-04-02 05:54:07,186 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
Sentence[12]: "Do you work in mumbai google John and then went to Washington" → ["mumbai"/LOC, "John"/PER, "Washington"/LOC]
The following NER tags are found:
Span[4:5]: "mumbai" → LOC (0.9994)
Span[6:7]: "John" → PER (0.9999)
Span[

In [ ]:
sentence2 = Sentence("Hi how are you?")
tagger.predict(sentence2)
ner = ""
print(sentence2.labels)
for i in sentence2.labels:

  ner+=i.data_point.text+" "+i.value+","
ner

[]


''

In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:

# sample sentences
sentences = [
    "I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world."    "This movie was terrible. I hated every moment of it.",
    "The weather is beautiful today!",
    "I feel really sad and depressed lately.",
    "This food is delicious. I could eat it every day."
]

sia = sia()
# perform sentiment analysis on each sentence and classify as positive or negative



In [ ]:
def sentiment(sentence):
    scores = sia.polarity_scores(sentence)
    
    if scores['compound'] > 0:
        sentiment = 'positive'
    elif scores['compound'] < 0:
        sentiment = 'negative'
    else:
        sentiment = 'neutral'
 
    return sentiment

In [ ]:
sentiment("This food is delicious. I could eat it every day.")

'positive'

In [ ]:
ss = "This food is delicious. I could eat it every day.This food is delicious. I could eat it every day.This food is delicious. I could eat it every day.This food is delicious. I could eat it every day.This food is delicious. I could eat it every day.This food is delicious. I could eat it every day.This food is delicious. I could eat it every day.This food is delicious. I could eat it every day."
ss[:150]

'This food is delicious. I could eat it every day.This food is delicious. I could eat it every day.This food is delicious. I could eat it every day.Thi'

In [ ]:
from torch.utils.data import Dataset
import json

class ChatData(Dataset):
    def __init__(self, path:str, tokenizer):
        self.data = json.load(open(path, "r"))

        self.X = []
        for i in self.data.values():
            self.X.append(i['query'][:150])
            self.X.append(i['reply'][:150])

        for idx, i in enumerate(self.X):
            # print(idx,i)
            # if idx == 1000:
            #     break
            try:
                senti = sentiment(i)
                s = Sentence(i)
                tagger.predict(s)
                ner = ""
                for x in s.labels:
                    ner+=x.data_point.text+" "+x.value+","
                self.X[idx] = "<startofstring> "+i+" <reply>: "+self.X[idx+1]+" <endofstring>"+" \n<ner>: "+ner+" \n<sentiment>: "+senti
            except:
                break

        self.X = self.X[:50000]
        
        print(self.X[0])

        self.X_encoded = tokenizer(self.X,max_length=45, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

In [ ]:
optim = Adam(model.parameters(), lr=1e-3)

In [9]:
def train(chatData, model, optim):

    epochs = 12

    for i in tqdm(range(epochs)):
        for X, a in chatData:
            X = X.to(device)
            a = a.to(device)
            optim.zero_grad()
            loss = model(X, attention_mask=a, labels=X).loss
            loss.backward()
            optim.step()
        torch.save(model.state_dict(), "/content/drive/MyDrive/NLP-Project/model_state_ner_sent2.pt")
        print(infer("hello how are you"))
        print(infer("do you love music"))

def infer(inp):
    inp = "<startofstring> "+inp+" <reply>: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)
    output = model.generate(X, attention_mask=a )
    output = tokenizer.decode(output[0])
    return output

In [ ]:
chatAll = ChatData("/content/drive/MyDrive/NLP-Project/df_all_json.json", tokenizer)
chatAll = DataLoader(chatAll, batch_size=64)

<startofstring> Hello How are you doing today? <reply>: whats up MD im doing good how are you doing? <endofstring> 
<ner>:  
<sentiment>: neutral


In [ ]:
import pickle

with open('chat_all.pkl', 'wb') as file:  
    pickle.dump(chatAll, file)

with open('chat_all.pkl', 'rb') as handle:
    chatalldata = pickle.load(handle)

In [ ]:
len(chatalldata.dataset)

In [ ]:
model.train()
print("training .... ")
train(chatAll, model, optim)

print("infer from model : ")
while True:
  inp = input()
  print(infer(inp))

training .... 


  0%|          | 0/12 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<startofstring> hello how are you <reply>: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


  8%|▊         | 1/12 [10:32<1:55:55, 632.35s/it]

<startofstring> do you love music <reply>: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
100%|██████████| 12/12 [2:07:22<00:00, 636.86s/it]


<startofstring> hello how are you <reply>: Doing pretty good, how about yourself? How are you? <pad>
<startofstring> do you love music <reply>: I love to play piano to! I play piano and ukule
infer from model : 
do you like food


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<startofstring> do you like food <reply>: I love the smell of pine trees. There's nothing to do and


KeyboardInterrupt: ignored

In [11]:
device = 'cpu'

In [7]:
model.load_state_dict(torch.load('/content/drive/MyDrive/NLP-Project/model_state_ner_sent2.pt',map_location=torch.device('cpu')))

<All keys matched successfully>

In [12]:
print("infer from model : ")
while True:
  inp = input()
  print(infer(inp))

infer from model : 
hi


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<startofstring> hi <reply>: Hey! How's it going? <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
how are you


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<startofstring> how are you <reply>: I'm good. Same as last time around here, Oregon. I'm
do like to sing


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<startofstring> what do you know about china <reply>: My friend tagged me in their facebook page and I was
do you use facebook


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<startofstring> do you use facebook <reply>: yes. I do. I also like to use it because it's


KeyboardInterrupt: ignored

#Bleu Score

In [ ]:
model.config.n_positions

In [ ]:
queries = ['hi','hello','how are you','do you like music','what are you learning','thank you','do you like ramen','what is your favorite food','I dont talk to my friend','who is donald trump']
references = [['Hi!','Hi','hey','hello','Hi there!'],["I'm doing pretty alright","I'm doing great!","I'm doing great! How about you?"]]



In [ ]:
queries = df_all['query'][:50000].sample(n=1000, random_state=1)

In [ ]:
references = list(df_all['reply'][:50000].sample(n=1000, random_state=1))
references = [x[:150] for x in references]

In [ ]:
references = []
hypothesis = []
try:
  for ind in queries.index:
    s = queries[ind][:150]
    if type(s) == str:
      inf = infer(s)
      hypothesis.append(inf)
      references.append(queries[ind])
except:
  pass


In [ ]:
import pickle

In [ ]:
with open('hypo.pkl', 'wb') as file:
      
    # A new file will be created
    pickle.dump(hypothesis, file)

In [ ]:

with open('hypo.pkl', 'rb') as handle:
    hypothes = pickle.load(handle)

In [ ]:
hypothes

In [ ]:

hypothes = []
for x in hypothesis:
  x = x.split('<reply>:')
  x = x[1]
  hypothes.append(x)
  

In [ ]:
print(list(queries))
print(hypothes)
print(references)


["I've seen some fun youtube videos with AI generated music", 'Take a wild guess.', 'I love that scene too. Its just so funny cause kip gets the girl in the end! Haha', 'haha yeah exactly', 'I might try to stop by', 'What would the name of that class be?', "That's so nice! I'm from Sao Paulo", 'HAHAHA, that was a very good pun', 'Hahaha I wish Netflix would make a show that brings dietitians and doctors, to read or watch those bloggers posting about their "diets" and evaluating that. I think there is so many things people say nowadays on the internet, and those things propagate really fast because everyone can see it in a matter of seconds and share it with the click of a button.', 'Man where do they get these questions? Haha I swear it\'s out of a book called "awkward questions to ask on a first date" haha Hmm. What would I do... I would definitely make food with it!', 'they were playing smash in the library', 'Aw stink', 'Interesting, interesting.', 'Yo, how did you invite me? Kyle?'

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
scores = []
for i in range(1000):
  ref = references[i].split()
  hyp = hypothes[i].split()

  score = sentence_bleu(ref, hyp)
  scores.append(score)


In [ ]:
(sum(scores)/1000)*100

4.1996903206846415e-230

In [ ]:
pip install evaluate

In [ ]:
pip install bert_score

In [ ]:
from evaluate import load

In [ ]:
bertscore = load("bertscore")

results = bertscore.compute(predictions=hypothes, references=references, lang="en")

In [ ]:
results

{'precision': [0.8739576935768127,
  0.7637505531311035,
  0.8219268321990967,
  0.8291142582893372,
  0.9096224308013916,
  0.8515905737876892,
  0.8366804718971252,
  0.8194328546524048,
  0.8244762420654297,
  0.8152113556861877,
  0.825177788734436,
  0.8258952498435974,
  0.8738341331481934,
  0.8339178562164307,
  0.8271377682685852,
  0.8415591716766357,
  0.8238868713378906,
  0.823434054851532,
  0.819703996181488,
  0.8157368302345276,
  0.8317818641662598,
  0.8205695748329163,
  0.8133112192153931,
  0.8185844421386719,
  0.8453844785690308,
  0.8227843046188354,
  0.8193209171295166,
  0.8121696710586548,
  0.8177536129951477,
  0.8235093355178833,
  0.8250522613525391,
  0.8149885535240173,
  0.8918286561965942,
  0.8414329290390015,
  0.8164869546890259,
  0.8405522108078003,
  0.8175331354141235,
  0.8278588056564331,
  0.8203345537185669,
  0.8068559169769287,
  0.7945468425750732,
  0.8265459537506104,
  0.7690318822860718,
  0.8401925563812256,
  0.8300396203994751,


#Training with Empathetic data

In [ ]:
from torch.utils.data import Dataset
import json

class EmpData(Dataset):
    def __init__(self, path:str, tokenizer):
        self.data = json.load(open(path, "r"))

        self.X = []
        for i in self.data.values():
            self.X.append(i['user1'])
            self.X.append(i['user2'])

        for idx, i in enumerate(self.X):
            try:
                self.X[idx] = "<startofstring> "+i+" <reply>: "+self.X[idx+1]+" <endofstring>"
            except:
                break

        # self.X = self.X[:10000]
        
        print(self.X[0])

        self.X_encoded = tokenizer(self.X,max_length=40, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

In [ ]:
empData = EmpData("/content/drive/MyDrive/IR_Project4/empathetic_processed_json.json", tokenizer)
empData =  DataLoader(empData, batch_size=64)

<startofstring> I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world. <reply>: Was this a friend you were in love with_comma_ or just a best friend? <endofstring>


In [ ]:
model.train()
print("training .... ")
train(empData, model, optim)

print("infer from model : ")
while True:
  inp = input()
  print(infer(inp))

training .... 


  0%|          | 0/12 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
  8%|▊         | 1/12 [00:24<04:25, 24.10s/it]

<startofstring> hello how are you <reply>: I am sorry to hear that. <endofstring> <pad> <pad> <pad> <pad> <pad> <pad>
<startofstring> do you love music <reply>: I love music. <endofstring> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 17%|█▋        | 2/12 [00:49<04:06, 24.65s/it]

<startofstring> hello how are you <reply>: Hi how are you <endofstring> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<startofstring> do you love music <reply>: Do you love music? <endofstring> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<startofstring> hi <reply>: hi <endofstring> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
hello


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<startofstring> hello <reply>: Funny - we just built a house where we used to go camping when I was


KeyboardInterrupt: ignored

In [ ]:
path = "/content/drive/MyDrive/IR_Project4/chitchat_csv_json.json"
trydata = json.load(open(path, "r"))
cnt = 0
dele = []
for k,i in zip(trydata.keys(),trydata.values()):
  if i['reply'] == 'Ok Bye!':
    # print(k,i)
    dele.append(k)
    # del trydata[str(c)]

    cnt+=1
    
    
cnt

7045

In [ ]:
len(dele)

7045

In [ ]:
for i in dele:
  if i in trydata.keys():
    del trydata[i]

In [ ]:
ccnt = 0
for c,i in enumerate(trydata.values()):
  if i['reply'] == 'Ok Bye!':

    ccnt+=1
      
ccnt

0

In [ ]:
with open("/content/drive/MyDrive/IR_Project4/chitchat_csv_json_removed.json", "w") as outfile:
    json.dump(trydata, outfile)

Empathetic

In [ ]:
df_emp = pd.read_csv('/content/drive/MyDrive/IR_Project4/empathetic_processed.csv')
df_emp

In [ ]:
df_reddit = pd.read_csv('/content/drive/MyDrive/IR_Project4/reddit_to_index.csv')
df_reddit.tail(30)

In [ ]:
df_reddit = df_reddit[['body','parent_body']]

In [ ]:
df_reddit.tail()

,body,parent_body
68486,Not a structural guy. ME (sitting for PE this ...,I personally have a mechanical engineering deg...
68487,SLO and UCSD does. It's a program with SE focu...,What's an SE degree? I might tell a layperson ...
68488,There’s a couple of programs that are SE focus...,What's an SE degree? I might tell a layperson ...
68489,I have an master’s in architecture in addition...,I personally have a mechanical engineering deg...
68490,What's an SE degree? I might tell a layperson ...,I personally have a mechanical engineering deg...


In [ ]:
df_reddit = df_reddit.rename(columns={'parent_body':'query','body':'reply'})

In [ ]:
df_emp = df_emp.rename(columns={"user1": "query", "user2": "reply"})

In [ ]:
df_emp = df_emp.drop(['context'], axis = 1)

In [ ]:
df = df[['query','reply']]

In [ ]:
df_all = pd.concat([df,df_emp,df_reddit])


In [ ]:
df_all.reset_index(drop = True)

,query,reply
0,Hello How are you doing today?,whats up MD im doing good how are you doing?
1,"Im alright, I just took a nap. But it was one ...",oh wow haha so you still feel tired huh?
2,Yeah,did you go to bed late?
3,I have more of a head ache than anything else,dude thats terrible
4,"No, Its just adjusting to a time change But en...",im refining my skills on Sketch App have you h...
...,...,...
136467,I personally have a mechanical engineering deg...,Not a structural guy. ME (sitting for PE this ...
136468,What's an SE degree? I might tell a layperson ...,SLO and UCSD does. It's a program with SE focu...
136469,What's an SE degree? I might tell a layperson ...,There’s a couple of programs that are SE focus...
136470,I personally have a mechanical engineering deg...,I have an master’s in architecture in addition...


In [ ]:
df_all.to_csv('/content/drive/MyDrive/NLP-Project/df_all.csv') 

In [ ]:
df_all = pd.read_csv('/content/drive/MyDrive/NLP-Project/df_all.csv')


CSV > JSON


In [ ]:

import csv
import json
 

def make_json(csvFilePath, jsonFilePath):
     
    data = {}
     
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        i = 0
        for rows in csvReader:
             
            key = str(i)
            data[key] = rows
            i+=1

    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))
         

In [ ]:
csvFilePath = r'/content/drive/MyDrive/NLP-Project/df_all.csv'
jsonFilePath = r'/content/drive/MyDrive/NLP-Project/df_all_json.json'
 
make_json(csvFilePath, jsonFilePath)